In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sp

## Things which are given / can be calculated 

In [17]:
P_d = sp.symbols('P_d')
D_p, D_g = sp.symbols('D_p D_g')
N_p, N_g = sp.symbols('N_p N_g') 
C, F = sp.symbols('C F')
n_p, n_g, n_g_guess = sp.symbols('n_p n_g n_g_guess')
VR, VR_guess, v_t = sp.symbols('VR VR_guess v_t')
power, lifetime = sp.symbols('power lifetime')
temp = sp.symbols('temp')


#enter given values

all_values = {
    P_d: 10,
    D_p: sp.nan,
    D_g: sp.nan,
    C: sp.nan,
    N_p: 19,
    N_g: sp.nan,
    n_p: 380,
    n_g: sp.nan,
    n_g_guess: 387,
    VR: sp.nan,
    VR_guess: sp.nan,
    v_t: sp.nan,
    F: sp.nan,
    power: sp.nan,
    lifetime: sp.nan
}

all_values, len(all_values)


Things to find from a table

In [4]:
J_p = None
J_g = None


#some of these might be equations
k_o = None
k_s = None
k_m = None
k_B = None
k_v = None
k_R = None

safety_factor = 1


## Prelim calculations
VR, N_g, D_g, P_d etc

### Create the equations

In [22]:

eq1 = sp.Eq(VR_guess, n_p / n_g_guess)

eq2_do_not_use = sp.Eq(N_g, N_p * VR_guess)

# eq2_5 = sp.Eq(temp, round(N_p * VR_guess, 0))

# eq2 = eq2_do_not_use.subs(temp, round(, 0))

eq3 = sp.Eq(VR, N_g / N_p)
eq4 = sp.Eq(n_g, n_p / VR)
eq5 = sp.Eq(D_p, N_p / P_d)
eq6 = sp.Eq(D_g, N_g / P_d)
eq7 = sp.Eq(C, (N_p + N_g) / (2 * P_d))
eq8 = sp.Eq(v_t, np.pi * D_p * n_p / 12)



### Solve the equations

In [13]:

known_values = {}

for var, value in all_values.items():
    if value is not sp.nan:
        known_values[var] = value

# Solve equations with known values
solutions = sp.solve([eq1, eq3, eq4, eq5, eq6, eq7, eq8], dict=True)

# Update known values with solutions
for solution in solutions:
    for var, expr in solution.items():
        if expr is not None:
            known_values[var] = expr.subs(known_values)
            

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")






P_d = 10
N_p = 0.10051891142646*v_t
n_p = 380
n_g_guess = 387
C = 0.00502594557132302*v_t*(n_g + 380)/n_g
D_g = 3.81971863420549*v_t/n_g
D_p = 0.010051891142646*v_t
N_g = 38.1971863420549*v_t/n_g
VR = 380/n_g
VR_guess = 380/387


### Solve the equations multiple times bc some of them depends on others

In [24]:
# Iteratively solve equations until all variables are found

run_count = 0

while True:
    previous_known_values = known_values.copy()
    
    # Solve equations with current known values
    solutions = sp.solve([eq1, eq3, eq4, eq5, eq6, eq7, eq8], dict=True)
    
    # Update known values with solutions
    for solution in solutions:
        for var, expr in solution.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)
    
    # Break if no new values are found
    if known_values == previous_known_values:
        break
    else:
        run_count += 1

    if run_count > 5:
        break

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")


P_d = 10
N_p = 19
n_p = 380
n_g_guess = 387
C = 0.00502594557132302*v_t*(n_g + 380)/n_g
D_g = 3.81971863420549*v_t/n_g
D_p = 0.010051891142646*v_t
N_g = 38.1971863420549*v_t/n_g
VR = 380/n_g
VR_guess = 380/387


## Force calculations

## Calulating correction Factors

## Loading Cycles
Z_**, Y_**

## Stresses and allowable Stresses

## Brinell Hardness Calculations